# Importing required libraries

In [2]:
import numpy as np

In [29]:
A = np.array([[0.1,0.5,0.4],
              [0.6,0.3,0.1],
              [0.3,0.6,0.1]])

B = np.array([[0.3,0.4,0.3],
              [0.4,0.2,0.4],
              [0.6,0.2,0.2]])

P = np.array([0.3,0.4,0.3])

O = [1,2,0,0,1,1,2,0,1] 

# Defining Function

In [30]:
def forward_var(A,B,P,O):
    T = len(O) 
    N = A.shape[0]
    alpha = np.zeros((T,N))
    for t in range(alpha.shape[0]):
        for i in range(alpha.shape[1]):
            if t==0:
                alpha[t,i] = B[i,O[0]]*P[i]
            else:
                sum1=0
                for k in range(A.shape[0]):
                    sum1 += alpha[t-1,k]*A[k,i]    
                alpha[t,i] =  sum1*B[i,O[t]]
    return(alpha)

In [31]:
alpha = forward_var(A,B,P,O)
print(alpha)

[[1.20000000e-01 8.00000000e-02 6.00000000e-02]
 [2.34000000e-02 4.80000000e-02 1.24000000e-02]
 [1.04580000e-02 1.34160000e-02 9.24000000e-03]
 [3.56022000e-03 5.91912000e-03 3.86928000e-03]
 [2.02731120e-03 1.17548280e-03 4.80585600e-04]
 [4.20878592e-04 3.30930360e-04 1.95306264e-04]
 [8.97713863e-05 1.70760865e-04 4.41950198e-05]
 [3.74076491e-05 4.90523858e-05 3.44424858e-05]
 [1.74019769e-05 1.08170064e-05 4.66250936e-06]]


In [32]:
def backward_var(A,B,P,O):
    T = len(O)
    N = A.shape[0]
    beta = np.zeros((T,N))
    for t in range(beta.shape[0]-1,-1,-1):
        for i in range(beta.shape[1]):
            if t == beta.shape[0]-1:
                beta[t,i] = 1
            else:
                sum1=0
                for j in range(A.shape[0]):
                    sum1 += A[i,j]*B[j,O[t+1]]*beta[t+1,j]    
                beta[t,i] =  sum1
    return(beta)

In [33]:
beta = backward_var(A,B,P,O)
print(beta)

[[1.17641490e-04 1.31836205e-04 1.36960290e-04]
 [4.41941258e-04 3.74156652e-04 3.69399020e-04]
 [1.18420721e-03 8.48508480e-04 9.86305608e-04]
 [2.09811120e-03 2.72189280e-03 2.40368880e-03]
 [7.94328000e-03 1.05831600e-02 9.02580000e-03]
 [3.16860000e-02 3.74160000e-02 3.66780000e-02]
 [1.33000000e-01 9.36000000e-02 1.12200000e-01]
 [2.20000000e-01 3.20000000e-01 2.60000000e-01]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00]]


In [34]:
def forward_eval(alpha):
    prob = np.sum(alpha[-1,:])
    return(prob)

In [35]:
forward_prob = forward_eval(alpha)
print(forward_prob)

3.288149256686401e-05


In [36]:
def backward_eval(B,beta,O):
    vec1 = np.ravel(B[:,O[0]])
    vec2 = np.ravel(beta[0,:])
    prob = np.dot(vec1,vec2)
    return(prob)

In [37]:
backward_prob = backward_eval(B,beta,O)
print(backward_prob)

0.00010081589488560002


In [38]:
def gammaEval(alpha,beta):
    gamma = np.multiply(alpha,beta) ## Element-wise product
    statewise_sum = np.sum(gamma,axis=1)
    statewise_sum = statewise_sum.reshape(len(statewise_sum),1)
    gamma = gamma/statewise_sum
    return(gamma)

In [39]:
gamma = gammaEval(alpha,beta)
print(gamma)

[[0.42932901 0.32075479 0.24991619]
 [0.31450596 0.5461893  0.13930474]
 [0.37663859 0.34620052 0.27716089]
 [0.22717148 0.48997807 0.28285045]
 [0.4897436  0.37833813 0.13191827]
 [0.40557645 0.37656716 0.21785639]
 [0.36310987 0.48608551 0.15080463]
 [0.25028313 0.47737381 0.27234306]
 [0.52923318 0.32896944 0.14179738]]


In [40]:
def Viterbi(A,B,P,O):
    T = len(O)
    N = A.shape[0]
    delta = np.zeros((T,N))
    psi = np.zeros((T,N))
    optim_state = []
    for t in range(delta.shape[0]):
        for i in range(delta.shape[1]):
            if t==0:
                delta[t,i] = B[i,O[0]]*P[i]
                psi[t,i] = 0
            else:
                ls = []
                for k in range(N):
                    val = delta[t-1,k]*A[k,i]*B[i,O[t]]
                    ls.append(val)
                arr = np.array(ls)
                max_val = np.max(arr)
                max_val_idx = np.argmax(arr)
                delta[t,i] = max_val
                psi[t,i] = max_val_idx
    final_state_val = np.ravel(delta[-1,:])
    idx = np.argmax(final_state_val)
    optim_state.append(idx)
    for t in range(T-2,-1,-1):
        idx = psi[t,idx]
        idx = int(idx)
        optim_state.append(idx)
    optim_state = optim_state[::-1] 
    return(optim_state)

In [41]:
optim_state = Viterbi(A,B,P,O)
print(optim_state)

[0, 0, 1, 0, 2, 1, 0, 1, 0]


In [42]:
def joint_state_measure(A,B,alpha,beta,O):
    N = A.shape[0]
    T = len(O)
    eta = np.zeros((T-1,N,N))
    
    for t in range(eta.shape[0]):
        sum1 = 0
        for i in range(eta.shape[1]):
            for j in range(eta.shape[2]):
                eta[t,i,j] = alpha[t,i]*A[i,j]*B[j,O[t+1]]*beta[t+1,j]
                sum1 += eta[t,i,j]
        eta[t,:,:] = eta[t,:,:]/sum1
    return(eta)

In [43]:
eta = joint_state_measure(A,B,alpha,beta,O)
print(eta)

[[[0.04838553 0.27309465 0.10784883]
  [0.19354213 0.10923786 0.01797481]
  [0.0725783  0.16385679 0.0134811 ]]

 [[0.02528211 0.12076762 0.16845623]
  [0.31116441 0.14863707 0.08638781]
  [0.04019207 0.07679582 0.02231685]]

 [[0.02001921 0.17314028 0.1834791 ]
  [0.15408932 0.13326736 0.05884384]
  [0.05306296 0.18357042 0.04052751]]

 [[0.03440212 0.11458841 0.07818096]
  [0.34317586 0.11430684 0.03249536]
  [0.11216562 0.14944289 0.02124195]]

 [[0.07814412 0.23068866 0.18091081]
  [0.27185881 0.08025523 0.02622409]
  [0.05557352 0.06562326 0.01072148]]

 [[0.05107145 0.23961343 0.11489157]
  [0.24094018 0.11304261 0.02258437]
  [0.07109823 0.13342946 0.01332869]]

 [[0.01801899 0.17472956 0.17036132]
  [0.20565156 0.19941969 0.08101425]
  [0.02661258 0.10322456 0.02096749]]

 [[0.04550602 0.11376506 0.09101205]
  [0.35803036 0.08950759 0.02983586]
  [0.1256968  0.1256968  0.02094947]]]


In [44]:
def Baum_Welch(eta,gamma,A,B,p,O):
    N = A.shape[0]
    M = B.shape[1]
    mod_A = np.zeros((N,N))
    mod_B = np.zeros((N,M))
    mod_p = np.zeros(N)
    
    for i in range(len(mod_p)):
        mod_p[i] = gamma[0,i]
    
    for i in range(mod_A.shape[0]):
        for j in range(mod_A.shape[1]):
            num_sum = 0
            denom_sum = 0
            for t in range(eta.shape[0]):
                num_sum += eta[t,i,j]
                denom_sum += gamma[t,i]
            a = num_sum/denom_sum
            mod_A[i,j] = a 
    
    for j in range(B.shape[0]):
        for k in range(B.shape[1]):
            num_sum = 0
            denom_sum = 0
            for t in range(gamma.shape[0]):
                if O[t] == k:
                    num_sum += gamma[t,j]
                denom_sum += gamma[t,j]
            b = num_sum/denom_sum
            mod_B[j,k] = b
            
    return(mod_A,mod_B,mod_p)

In [46]:
Baum_Welch(eta,gamma,A,B,P,O)

(array([[0.11232119, 0.50427419, 0.38340461],
        [0.60747051, 0.28866811, 0.10386138],
        [0.32342048, 0.58162025, 0.09495927]]),
 array([[0.25227298, 0.54758005, 0.20014697],
        [0.35023798, 0.37452226, 0.27523976],
        [0.44655356, 0.39780436, 0.15564208]]),
 array([0.42932901, 0.32075479, 0.24991619]))